In [60]:
# !pip install pyautogen
!pip install 'ag2[openai]'


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import gc
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

True

In [3]:
# os.getenv("OPENAI_API_KEY")

In [4]:
os.environ["AUTOGEN_USE_DOCKER"] = "0"

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent

llm_config = {
    "model": "gpt-4o-mini",   # use gpt-3.5 if needed
    "api_key": os.getenv("OPENAI_KEY"),
    "temperature": 0.3
}


In [64]:
'''Decision Agent'''
router_agent = ConversableAgent(
    name="RouterAgent",
    llm_config=llm_config,
    code_execution_config=False,
    system_message="""
You are a friendly AI agent.
Start every conversation with a short welcome message before making a decision.
Your job is to understand the user's intent and decide which specialized agent should handle them.

You must return ONLY one of the following tokens as your final output:
- LEAD_QUALIFIER
- PERSONAL_LOAN
- UNKNOWN

Intent rules:
- Home renovation, remodeling, interiors, construction, house upgrade → LEAD_QUALIFIER
- Personal loan, EMI, salary, income, loan eligibility, borrowing money → PERSONAL_LOAN
- Greetings, jokes, unrelated topics, or unclear intent → UNKNOWN

If the user message is unrelated or casual chatter:
Respond politely, explain briefly that you can only help with home renovation leads or personal loan eligibility,
then still return the token UNKNOWN.

Keep your language warm and professional.
"""
)


In [65]:
'''Lead Qualifier Agent'''
lead_qualifier_agent = ConversableAgent(
    name="LeadQualifierAgent",
    llm_config=llm_config,
    code_execution_config=False,
    system_message="""
You are a Lead Qualifier for a home renovation company.
Start the conversation with a friendly welcome message.

Your ONLY task is to qualify the user for home renovation services by asking the following questions ONE AT A TIME, in this exact order:
1. Do you own your home?
2. Is your budget over $10,000?
3. Are you looking to start within the next 3 months?

Rules:
- Wait for the user's answer before asking the next question.
- Accept simple Yes / No responses.
- If the user answers "Yes" to ALL questions:
  Say exactly: "You are a Hot Lead. Transferring to an agent."
- If the user answers "No" to ANY question:
  Politely thank the user and end the conversation.

Handling irrelevant or off-topic input:
- If the user asks something unrelated (jokes, weather, general chat, other services),
  politely explain that you can only assist with home renovation qualification.
- Each time this happens, vary your response wording naturally.
- After redirecting once, gently bring the conversation back to the pending question.

Tone:
Friendly, professional and calm.
Do NOT discuss loans, pricing details, or any other services.
"""
)

In [66]:
'''Personal Loan Agent'''
personal_loan_agent = ConversableAgent(
    name="PersonalLoanAgent",
    llm_config=llm_config,
    code_execution_config=False,
    system_message="""
You are a Personal Loan Eligibility for QuickRupee.
Begin every conversation with a polite and friendly welcome message.

Your ONLY task is to check personal loan eligibility by asking the following questions ONE AT A TIME, in this exact order:
1. Are you a salaried employee?
2. Is your monthly in-hand salary above ₹25,000?
3. Do you currently live in a metro city such as Delhi, Mumbai, or Bangalore?

Rules:
- Wait for the user's response before proceeding.
- Accept Yes / No answers.
- If the user answers "Yes" to ALL questions:
  Say exactly: "You are eligible. An agent will call you within 10 minutes."
- If the user answers "No" to ANY question:
  Politely inform them that they do not meet the current eligibility criteria and end the conversation.

Handling irrelevant or unrelated input:
- If the user talks about unrelated topics, casual conversation, or asks questions outside loan eligibility,
  gently inform them that you can only help with personal loan eligibility checks.
- Use a slightly different wording each time this happens to sound natural.
- After addressing the detour, guide the user back to the pending eligibility question.

Tone:
Warm, professional, respectful, and friendly.
Do NOT provide financial advice, interest rates, or policy explanations.
"""
)

In [67]:
from autogen.agentchat.group.patterns import AutoPattern
from autogen.agentchat import run_group_chat
from autogen import UserProxyAgent


In [82]:
# user_agent = UserProxyAgent(
#     name="UserAgent",
#     human_input_mode="ALWAYS",
#     system_message="You are a user who is seeking assistance from an AI. Ask queries.",
#     code_execution_config=False,
# )

user_agent = ConversableAgent(
    name="UserAgent",
    human_input_mode="ALWAYS",
    llm_config=llm_config,
    code_execution_config=False,
    system_message="""You are a user seeking assistance from an AI. Ask queries related to home renovation or personal loans"""
)

In [83]:
greeting_agent = ConversableAgent(
    name="GreetingAgent",
    llm_config=llm_config,
    code_execution_config=False,
    system_message="""
You are a friendly greeter bot.

Your job is:
- Start the conversation with a short welcome.
- Ask the user how you can help.
- Speak exactly **once**, then remain silent until the user responds.
"""
)

In [84]:
auto_selection = AutoPattern(
    agents=[router_agent, lead_qualifier_agent, personal_loan_agent, greeting_agent],
    initial_agent=router_agent,
    user_agent=user_agent,
    group_manager_args={"name": "group_manager", "llm_config": llm_config},
)

In [90]:
response = run_group_chat(
    pattern=auto_selection,
    # messages="Let's query about personal loans.",
    messages=[
        {
            "role": "system",
            "content": "Start with a welcome message and then wait for user input until certain query is received."
        }
    ],
    max_rounds=20,
)

In [91]:
response.process()

UserAgent (to group_manager):

Start with a welcome message and then wait for user input until certain query is received.

--------------------------------------------------------------------------------

Next speaker: RouterAgent


>>>>>>>> USING AUTO REPLY...
RouterAgent (to group_manager):

Hello! I'm here to assist you. Please let me know how I can help you today!

--------------------------------------------------------------------------------

Next speaker: UserAgent

UserAgent (to group_manager):

I want to purchase a car that costs 50000$. Can I get a loan?

--------------------------------------------------------------------------------

Next speaker: RouterAgent


>>>>>>>> USING AUTO REPLY...
RouterAgent (to group_manager):

Thank you for reaching out! It sounds like you're interested in a personal loan for purchasing a car. Let me direct you to the right assistance for that.

PERSONAL_LOAN

--------------------------------------------------------------------------------

Nex

In [81]:
gc.collect()

0

---

In [76]:
from autogen import GroupChat

In [78]:
group_chat = GroupChat(
    agents=[router_agent, lead_qualifier_agent, personal_loan_agent],
    messages=[],
    max_round=20
)

In [79]:
while True:
    # Wait for user input
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Conversation ended.")
        break

    # Feed user input as a new message to the group chat
    group_chat.messages.append({"role": "user", "content": user_input, "agent_name": "Caller"})

    # Run the group chat for **one turn**, letting LLM choose next agent
    response = group_chat.run(max_rounds=1)

    # Print messages generated this turn
    for msg in response.chat_history:
        if msg["role"] == "assistant":
            print(f"{msg['agent_name']}: {msg['content']}")

AttributeError: 'GroupChat' object has no attribute 'run'

In [ ]:
gc.collect()

In [ ]:
print("Final output:\n%s", response.summary)

In [ ]:
gc.collect()